# Bond Debt Service Example

This notebook demonstrates how to use PyProforma v2's debt line items to model bond debt service with:
- Multiple bond issuances across different years
- Automatic calculation of principal and interest using level annual debt service
- Tracking total debt service and outstanding balances
- Using tags to aggregate debt-related items

In [1]:
from pyproforma.v2 import FixedLine, FormulaLine, ProformaModel, create_debt_lines

## Creating Debt Lines Without the Factory (Manual Approach)

Before we show the full model using the convenient `create_debt_lines()` factory, let's demonstrate how to create debt line items manually. This shows the underlying steps:

1. Import the individual classes: `DebtCalculator`, `DebtPrincipalLine`, `DebtInterestLine`
2. Create a shared `DebtCalculator` instance
3. Create separate principal and interest line items that both reference the same calculator

This manual approach gives you full control over the debt line configuration.

In [2]:
from pyproforma.v2 import DebtCalculator, DebtPrincipalLine, DebtInterestLine

# Step 1: Create a shared calculator
# This manages the debt schedules and amortization calculations
calculator = DebtCalculator(
    par_amounts_line_item="bond_proceeds",  # Which line item to check for new issuances
    interest_rate=0.05,  # 5% annual interest rate
    term=10,  # 10-year amortization period
)

# Step 2: Create principal line item
# This line item will return principal payments each year
principal_manual = DebtPrincipalLine(
    calculator=calculator,  # Share the calculator
    label="Principal Payment (Manual)",
    tags=["debt_service_manual"],
)

# Step 3: Create interest line item
# This line item will return interest payments each year
interest_manual = DebtInterestLine(
    calculator=calculator,  # Share the SAME calculator instance
    label="Interest Expense (Manual)",
    tags=["debt_service_manual"],
)

print("✓ Debt line items created manually")
print(f"  Calculator: {calculator}")
print(f"  Principal: {principal_manual}")
print(f"  Interest: {interest_manual}")
print()
print("Key point: Both principal and interest reference the SAME calculator")
print("           This ensures they use identical debt schedules.")

✓ Debt line items created manually
  Calculator: <pyproforma.v2.line_items.debt_line.DebtCalculator object at 0x7f83a8ac7dd0>
  Principal: DebtPrincipalLine(par_amounts_line_item='bond_proceeds', interest_rate=0.05, term=10, label='Principal Payment (Manual)')
  Interest: DebtInterestLine(par_amounts_line_item='bond_proceeds', interest_rate=0.05, term=10, label='Interest Expense (Manual)')

Key point: Both principal and interest reference the SAME calculator
           This ensures they use identical debt schedules.


### Comparison: Factory Function vs Manual

The `create_debt_lines()` factory function does the above steps automatically:

```python
# This single line replaces the manual steps above:
principal, interest = create_debt_lines(
    par_amounts_line_item="bond_proceeds",
    interest_rate=0.05,
    term=10,
    principal_label="Principal Payment",
    interest_label="Interest Expense",
    tags=["debt_service"],
)
```

**When to use manual approach:**
- You need fine-grained control over calculator configuration
- You want to share a calculator across multiple models
- You're creating custom debt workflows

**When to use factory function:**
- Standard debt modeling (most common case)
- Cleaner, more concise code
- Guaranteed consistency between principal and interest

## Define the Debt Service Model

The model includes:
- Bond proceeds (issuances in 2024, 2026, and 2028)
- Principal and interest payments (using `create_debt_lines` factory)
- Total debt service calculation
- Operating metrics and debt service coverage ratio

In [3]:
class DebtServiceModel(ProformaModel):
    """
    A financial model with bond debt service.

    This model demonstrates:
    - Multiple bond issuances across different years
    - Automatic calculation of principal and interest using level debt service
    - Tracking total debt service and outstanding balances
    - Using tags to aggregate debt-related items
    """

    # Define bond issuances
    bond_proceeds = FixedLine(
        values={year: 0 for year in range(2024, 2039)},  # Initialize all to 0
        label="Bond Proceeds",
        tags=["financing"],
    )
    # Override specific years with issuances
    bond_proceeds.values[2024] = 1_000_000  # $1M bond issued in 2024
    bond_proceeds.values[2026] = 500_000  # $500K bond issued in 2026
    bond_proceeds.values[2028] = 750_000  # $750K bond issued in 2028

    # Create debt line items using the factory function
    # This ensures principal and interest share the same debt schedules
    principal_payment, interest_expense = create_debt_lines(
        par_amounts_line_item="bond_proceeds",
        interest_rate=0.05,  # 5% annual interest rate
        term=10,  # 10-year amortization
        principal_label="Principal Payment",
        interest_label="Interest Expense",
        tags=["debt_service"],
    )

    # Calculate total debt service
    total_debt_service = FormulaLine(
        formula=lambda a, li, t: li.principal_payment[t] + li.interest_expense[t],
        label="Total Debt Service",
    )

    # Alternative: use tags to sum debt service
    total_debt_service_via_tags = FormulaLine(
        formula=lambda a, li, t: li.tag["debt_service"][t],
        label="Total Debt Service (via tags)",
    )

    # Calculate cumulative principal paid
    cumulative_principal_paid = FormulaLine(
        formula=lambda a, li, t: li.principal_payment[t]
        + (li.cumulative_principal_paid[t - 1] if t > 2024 else 0),
        label="Cumulative Principal Paid",
    )

    # Calculate debt service coverage ratio (example)
    revenue = FixedLine(
        values={year: 500_000 * (1.05 ** (year - 2024)) for year in range(2024, 2039)},
        label="Revenue",
    )

    operating_expenses = FormulaLine(
        formula=lambda a, li, t: li.revenue[t] * 0.6,
        label="Operating Expenses",
    )

    operating_income = FormulaLine(
        formula=lambda a, li, t: li.revenue[t] - li.operating_expenses[t],
        label="Operating Income",
    )

    debt_service_coverage_ratio = FormulaLine(
        formula=lambda a, li, t: (
            li.operating_income[t] / li.total_debt_service[t]
            if li.total_debt_service[t] > 0
            else 0
        ),
        label="Debt Service Coverage Ratio",
    )

In [4]:
# Helper functions for formatting output

def format_currency(value):
    """Format a value as currency."""
    return f"${value:,.0f}"

def format_ratio(value):
    """Format a value as a ratio."""
    return f"{value:.2f}x"

## Create the Model

We'll create a model for 15 years (2024-2038) to cover the full amortization of all three bonds:
- 2024 bond: 2024-2033
- 2026 bond: 2026-2035
- 2028 bond: 2028-2037

In [5]:
periods = list(range(2024, 2039))
model = DebtServiceModel(periods=periods)

print("Model created successfully!")
print(f"Periods: {periods[0]}-{periods[-1]}")

Model created successfully!
Periods: 2024-2038


## Bond Issuances

Let's review the bond issuances:

In [6]:
print("Bond Issuances:")
print(f"  2024: {format_currency(model.li.bond_proceeds[2024])}")
print(f"  2026: {format_currency(model.li.bond_proceeds[2026])}")
print(f"  2028: {format_currency(model.li.bond_proceeds[2028])}")

Bond Issuances:
  2024: $1,000,000
  2026: $500,000
  2028: $750,000


## Debt Service Schedule

View the debt service schedule for selected years:

In [7]:
print("Debt Service Schedule (Selected Years):")
print("-" * 80)
print(f"{'Year':<8} {'Principal':<15} {'Interest':<15} {'Total DS':<15} {'DSCR':<10}")
print("-" * 80)

selected_years = [2024, 2026, 2028, 2030, 2032, 2034, 2036, 2038]
for year in selected_years:
    principal = model.li.principal_payment[year]
    interest = model.li.interest_expense[year]
    total_ds = model.li.total_debt_service[year]
    dscr = model.li.debt_service_coverage_ratio[year]

    print(
        f"{year:<8} {format_currency(principal):<15} {format_currency(interest):<15} "
        f"{format_currency(total_ds):<15} {format_ratio(dscr):<10}"
    )

Debt Service Schedule (Selected Years):
--------------------------------------------------------------------------------
Year     Principal       Interest        Total DS        DSCR      
--------------------------------------------------------------------------------
2024     $79,505         $50,000         $129,505        1.54x     
2026     $127,406        $66,851         $194,257        1.14x     
2028     $200,094        $91,292         $291,385        0.83x     
2030     $220,603        $70,782         $291,385        0.92x     
2032     $243,215        $48,170         $291,385        1.01x     
2034     $138,640        $23,241         $161,881        2.01x     
2036     $88,098         $9,030          $97,128         3.70x     
2038     $0              $0              $0              0.00x     


## Key Observations

### 1. Level Debt Service
Each individual bond has constant annual payments (principal + interest):

In [8]:
ds_2024_bond_year1 = model.li.total_debt_service[2024]
print(f"2024 bond debt service (year 1): {format_currency(ds_2024_bond_year1)}")
print("(Note: Each individual bond has level debt service)")

2024 bond debt service (year 1): $129,505
(Note: Each individual bond has level debt service)


### 2. Interest Declining Over Time
As principal is paid down, interest decreases:

In [9]:
print("Interest payments over time:")
print(f"  2024 Interest: {format_currency(model.li.interest_expense[2024])}")
print(f"  2026 Interest: {format_currency(model.li.interest_expense[2026])}")
print(f"  2028 Interest: {format_currency(model.li.interest_expense[2028])}")
print(f"  2030 Interest: {format_currency(model.li.interest_expense[2030])}")
print("  (Interest declines as principal is paid down)")

Interest payments over time:
  2024 Interest: $50,000
  2026 Interest: $66,851
  2028 Interest: $91,292
  2030 Interest: $70,782
  (Interest declines as principal is paid down)


### 3. Principal Increasing Over Time
Note: 2026 and 2028 are higher due to multiple overlapping bonds:

In [10]:
print("Principal payments over time:")
print(f"  2024 Principal: {format_currency(model.li.principal_payment[2024])}")
print(f"  2026 Principal: {format_currency(model.li.principal_payment[2026])}")
print(f"  2028 Principal: {format_currency(model.li.principal_payment[2028])}")
print(f"  2030 Principal: {format_currency(model.li.principal_payment[2030])}")
print("  (Note: 2026 and 2028 are higher due to multiple overlapping bonds)")

Principal payments over time:
  2024 Principal: $79,505
  2026 Principal: $127,406
  2028 Principal: $200,094
  2030 Principal: $220,603
  (Note: 2026 and 2028 are higher due to multiple overlapping bonds)


### 4. Overlapping Bond Issues
In 2028, all three bonds are active:

In [11]:
peak_year = 2028
print(f"In {peak_year}, all three bonds are active:")
print(f"  - 2024 bond: year {peak_year - 2024 + 1} of 10")
print(f"  - 2026 bond: year {peak_year - 2026 + 1} of 10")
print(f"  - 2028 bond: year {peak_year - 2028 + 1} of 10")
print(f"Total debt service: {format_currency(model.li.total_debt_service[peak_year])}")

In 2028, all three bonds are active:
  - 2024 bond: year 5 of 10
  - 2026 bond: year 3 of 10
  - 2028 bond: year 1 of 10
Total debt service: $291,385


### 5. Final Bond Payment

In [12]:
last_year_with_debt = 2037  # 2028 + 10 - 1
print(f"Last payment in {last_year_with_debt}:")
print(f"  Principal: {format_currency(model.li.principal_payment[last_year_with_debt])}")
print(f"  Interest:  {format_currency(model.li.interest_expense[last_year_with_debt])}")
print(f"No debt service in 2038: {format_currency(model.li.total_debt_service[2038])}")

Last payment in 2037:
  Principal: $92,503
  Interest:  $4,625
No debt service in 2038: $0


### 6. Tag-based Aggregation
Verify that tag-based summation matches direct calculation:

In [13]:
tag_total = model.li.total_debt_service_via_tags[2028]
direct_total = model.li.total_debt_service[2028]
print(f"Total DS (direct):    {format_currency(direct_total)}")
print(f"Total DS (via tags):  {format_currency(tag_total)}")
print(f"Match: {abs(tag_total - direct_total) < 0.01}")

Total DS (direct):    $291,385
Total DS (via tags):  $291,385
Match: True


## Summary

This example demonstrates PyProforma v2's debt line items:
- **Easy setup**: Use `create_debt_lines()` to automatically generate principal and interest line items
- **Multiple issuances**: Handles overlapping bond issues automatically
- **Level debt service**: Each bond has constant annual payments
- **Flexible integration**: Works seamlessly with other line items and formulas
- **Tag support**: Aggregate related items using tags

## Debt Schedule Table


In [14]:
from pyproforma.v2.tables import ItemRow, LabelRow, BlankRow
template = [
    LabelRow(label="Debt Service Schedule", bold=True),
    BlankRow(),
    ItemRow(name="bond_proceeds"),
    ItemRow(name="principal_payment"),
    ItemRow(name="interest_expense"),
    ItemRow(name="total_debt_service"),
    BlankRow(),
    ItemRow(name="debt_service_coverage_ratio"),
]
model.tables.from_template(template, col_labels="Label").show()


Debt Service Schedule,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
Bond Proceeds,"1,000,000",0,"500,000",0,"750,000",0,0,0,0,0,0,0,0,0,0
Principal Payment,"79,505","83,480","127,406","133,776","200,094","210,098","220,603","231,633","243,215","255,376","138,640","145,572","88,098","92,503",0
Interest Expense,"50,000","46,025","66,851","60,480","91,292","81,287","70,782","59,752","48,170","36,009","23,241","16,309","9,030","4,625",0
Total Debt Service,"129,505","129,505","194,257","194,257","291,385","291,385","291,385","291,385","291,385","291,385","161,881","161,881","97,128","97,128",0
,,,,,,,,,,,,,,,
Debt Service Coverage Ratio,2,2,1,1,1,1,1,1,1,1,2,2,4,4,0
